# IPL Business Intelligence Analysis

This notebook focuses on creating detailed BI insights and reports from IPL data.

In [15]:
# Import required libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Window
import pyspark.sql.functions as F

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("IPL BI Analysis") \
    .master("local[*]") \
    .getOrCreate()

# Load the data
deliveries_df = spark.read.csv('data/deliveries.csv', header=True, inferSchema=True)
matches_df = spark.read.csv('data/matches.csv', header=True, inferSchema=True)

## 1. Per-Match Analysis

In [16]:
# Calculate per-match metrics
match_metrics = deliveries_df.groupBy('match_id', 'inning') \
    .agg(F.sum('total_runs').alias('innings_runs'),
         F.sum('is_wicket').alias('innings_wickets'),
         F.count(F.when(F.col('extras_type').isNotNull(), 1)).alias('extras_count'))

# Calculate runs per over
runs_per_over = deliveries_df.groupBy('match_id', 'inning', 'over') \
    .agg(F.sum('total_runs').alias('over_runs'))

# Calculate highest partnerships
partnerships = deliveries_df.groupBy('match_id', 'inning', 'batter', 'non_striker') \
    .agg(F.sum('total_runs').alias('partnership_runs')) \
    .groupBy('match_id', 'inning') \
    .agg(F.max('partnership_runs').alias('highest_partnership'))

print("\n--- Match Metrics Sample ---")
match_metrics.show(5)


--- Match Metrics Sample ---
+--------+------+------------+---------------+------------+
|match_id|inning|innings_runs|innings_wickets|extras_count|
+--------+------+------------+---------------+------------+
|  336009|     1|         187|              5|          12|
|  392196|     1|         165|              6|           7|
|  419132|     1|         163|             10|          10|
|  392196|     2|         169|              4|           6|
|  335993|     1|         147|              9|           8|
+--------+------+------------+---------------+------------+
only showing top 5 rows

+--------+------+------------+---------------+------------+
|match_id|inning|innings_runs|innings_wickets|extras_count|
+--------+------+------------+---------------+------------+
|  336009|     1|         187|              5|          12|
|  392196|     1|         165|              6|           7|
|  419132|     1|         163|             10|          10|
|  392196|     2|         169|              4

## 2. Per-Player Analysis

In [17]:
# Join with matches for season-wise analysis
player_stats = deliveries_df.join(matches_df, deliveries_df.match_id == matches_df.id)

# Batting statistics
batting_stats = player_stats.groupBy('season', 'batter') \
    .agg(F.sum('batsman_runs').alias('total_runs'),
         F.count('ball').alias('balls_faced'),
         F.count(F.when(F.col('batsman_runs') == 4, 1)).alias('fours'),
         F.count(F.when(F.col('batsman_runs') == 6, 1)).alias('sixes'),
         F.sum('is_wicket').alias('dismissals')) \
    .withColumn('average', F.round(F.col('total_runs') / F.col('dismissals'), 2)) \
    .withColumn('strike_rate', F.round(F.col('total_runs') * 100 / F.col('balls_faced'), 2))

print("\n--- Batting Statistics Sample ---")
batting_stats.orderBy(F.desc('total_runs')).show(5)


--- Batting Statistics Sample ---
+------+------------+----------+-----------+-----+-----+----------+-------+-----------+
|season|      batter|total_runs|balls_faced|fours|sixes|dismissals|average|strike_rate|
+------+------------+----------+-----------+-----+-----+----------+-------+-----------+
|  2016|     V Kohli|       973|        655|   84|   38|        12|  81.08|     148.55|
|  2023|Shubman Gill|       890|        582|   85|   33|        16|  55.63|     152.92|
|  2022|  JC Buttler|       863|        596|   84|   45|        15|  57.53|      144.8|
|  2016|   DA Warner|       848|        579|   88|   31|        16|   53.0|     146.46|
|  2024|     V Kohli|       741|        497|   62|   38|        13|   57.0|     149.09|
+------+------------+----------+-----------+-----+-----+----------+-------+-----------+
only showing top 5 rows

+------+------------+----------+-----------+-----+-----+----------+-------+-----------+
|season|      batter|total_runs|balls_faced|fours|sixes|dism

In [18]:
# Bowling statistics
bowling_stats = player_stats.groupBy('season', 'bowler') \
    .agg(F.sum('is_wicket').alias('wickets'),
         F.count('ball').alias('balls_bowled'),
         F.sum('total_runs').alias('runs_conceded'),
         F.count(F.when(F.col('batsman_runs') == 0, 1)).alias('dot_balls')) \
    .withColumn('economy', F.round(F.col('runs_conceded') * 6 / F.col('balls_bowled'), 2)) \
    .withColumn('dot_ball_percentage', F.round(F.col('dot_balls') * 100 / F.col('balls_bowled'), 2))

print("\n--- Bowling Statistics Sample ---")
bowling_stats.orderBy(F.desc('wickets')).show(5)


--- Bowling Statistics Sample ---
+-------+-----------+-------+------------+-------------+---------+-------+-------------------+
| season|     bowler|wickets|balls_bowled|runs_conceded|dot_balls|economy|dot_ball_percentage|
+-------+-----------+-------+------------+-------------+---------+-------+-------------------+
|   2021|   HV Patel|     35|         361|          461|      141|   7.66|              39.06|
|   2013|   DJ Bravo|     34|         392|          505|      157|   7.73|              40.05|
|   2013|JP Faulkner|     33|         395|          436|      186|   6.62|              47.09|
|2020/21|   K Rabada|     32|         414|          565|      175|   8.19|              42.27|
|   2023|  MM Sharma|     31|         268|          362|       90|    8.1|              33.58|
+-------+-----------+-------+------------+-------------+---------+-------+-------------------+
only showing top 5 rows

+-------+-----------+-------+------------+-------------+---------+-------+-----------

## 3. Head-to-Head Analysis

In [19]:
# Batsman vs Bowler analysis
head_to_head = deliveries_df.groupBy('batter', 'bowler') \
    .agg(F.sum('batsman_runs').alias('runs'),
         F.count('ball').alias('balls'),
         F.sum('is_wicket').alias('dismissals')) \
    .withColumn('strike_rate', F.round(F.col('runs') * 100 / F.col('balls'), 2))

print("\n--- Head to Head Analysis Sample ---")
head_to_head.orderBy(F.desc('balls')).show(5)


--- Head to Head Analysis Sample ---
+---------+---------+----+-----+----------+-----------+
|   batter|   bowler|runs|balls|dismissals|strike_rate|
+---------+---------+----+-----+----------+-----------+
|  V Kohli| R Ashwin| 179|  153|         1|     116.99|
|  V Kohli|RA Jadeja| 157|  148|         5|     106.08|
|RG Sharma|SP Narine| 143|  136|         9|     105.15|
|DA Warner|SP Narine| 195|  127|         2|     153.54|
| KL Rahul|JJ Bumrah| 150|  126|         3|     119.05|
+---------+---------+----+-----+----------+-----------+
only showing top 5 rows

+---------+---------+----+-----+----------+-----------+
|   batter|   bowler|runs|balls|dismissals|strike_rate|
+---------+---------+----+-----+----------+-----------+
|  V Kohli| R Ashwin| 179|  153|         1|     116.99|
|  V Kohli|RA Jadeja| 157|  148|         5|     106.08|
|RG Sharma|SP Narine| 143|  136|         9|     105.15|
|DA Warner|SP Narine| 195|  127|         2|     153.54|
| KL Rahul|JJ Bumrah| 150|  126|         

## 4. Match State Analysis

In [20]:
# Create window for running calculations
match_window = Window.partitionBy('match_id', 'inning').orderBy('over', 'ball')

# Calculate match state features
match_state = deliveries_df \
    .withColumn('current_score', F.sum('total_runs').over(match_window)) \
    .withColumn('wickets_in_hand', 10 - F.sum('is_wicket').over(match_window)) \
    .withColumn('balls_completed', F.count('*').over(match_window)) \
    .withColumn('run_rate', F.round(F.col('current_score') * 6 / F.col('balls_completed'), 2))

print("\n--- Match State Analysis Sample ---")
match_state.select('match_id', 'inning', 'over', 'current_score', 
                  'wickets_in_hand', 'run_rate').show(5)


--- Match State Analysis Sample ---
+--------+------+----+-------------+---------------+--------+
|match_id|inning|over|current_score|wickets_in_hand|run_rate|
+--------+------+----+-------------+---------------+--------+
|  335983|     1|   0|            0|             10|     0.0|
|  335983|     1|   0|            0|             10|     0.0|
|  335983|     1|   0|            1|             10|     2.0|
|  335983|     1|   0|            1|             10|     1.5|
|  335983|     1|   0|            5|             10|     6.0|
+--------+------+----+-------------+---------------+--------+
only showing top 5 rows

+--------+------+----+-------------+---------------+--------+
|match_id|inning|over|current_score|wickets_in_hand|run_rate|
+--------+------+----+-------------+---------------+--------+
|  335983|     1|   0|            0|             10|     0.0|
|  335983|     1|   0|            0|             10|     0.0|
|  335983|     1|   0|            1|             10|     2.0|
|  33598

In [21]:
# Stop Spark Session
spark.stop()